In [6]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

data = np.load('all_players_tensor.npy')

X = data[:, :, :]
X = np.delete(X, -2, axis=2)
y = data[:, :, -2]  # Labels: second last column indicating injury (1) or not (0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

y_train_aggregated = y_train[:, -1].unsqueeze(1)


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        return torch.sigmoid(out)  # Apply sigmoid activation


input_size = X.shape[2]  # Number of features
hidden_size = 128
num_layers = 4
output_size = 1

model = RNN(input_size, hidden_size, num_layers, output_size)

criterion = nn.BCELoss()  # Since we're using sigmoid activation in the model
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Training the model
num_epochs = 10

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)
        labels = labels[:, -1]  # Selecting only the last label in each sequence
        loss = criterion(outputs.squeeze(), labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')


# Evaluation of each match sequentially
with torch.no_grad():
    all_player_predictions = []
    for i in range(len(X_test)):
        player_predictions = []
        for j in range(X_test.size(1)):
            input_match = X_test[i:i + 1, j:j + 1, :]
            output = model(input_match)
            predicted = output.item()  # Directly take sigmoid value as prediction
            player_predictions.append(predicted)
        all_player_predictions.append(player_predictions)

    # Printing the sigmoid values
    for idx, player_preds in enumerate(all_player_predictions):
        print(f"Player {idx+1} sigmoid values: {player_preds}")


    accuracy = correct_predictions / total_predictions
    print(f'Accuracy on test set: {accuracy:.2f}')
print(all_player_predictions)


Epoch [1/10], Step [10/16], Loss: 0.5464
Epoch [2/10], Step [10/16], Loss: 0.2346
Epoch [3/10], Step [10/16], Loss: 0.1058
Epoch [4/10], Step [10/16], Loss: 0.0402
Epoch [5/10], Step [10/16], Loss: 0.2311
Epoch [6/10], Step [10/16], Loss: 0.2368
Epoch [7/10], Step [10/16], Loss: 0.0699
Epoch [8/10], Step [10/16], Loss: 0.1839
Epoch [9/10], Step [10/16], Loss: 0.2508
Epoch [10/10], Step [10/16], Loss: 0.1950
Player 1 sigmoid values: [0.3042445778846741, 0.08322568237781525, 0.4594651758670807, 0.04541587829589844, 0.5891686677932739, 0.3042445778846741, 0.3042445778846741, 0.14181369543075562, 0.3042445778846741, 0.08630217611789703, 0.4733598232269287, 0.5834811329841614, 0.3042445778846741, 0.3042445778846741, 0.3042445778846741, 0.3042445778846741, 0.3042445778846741, 0.3042445778846741, 0.08107483386993408, 0.45096707344055176, 0.055431168526411057, 0.039957575500011444, 0.06325232982635498, 0.1191403865814209, 0.0823129415512085, 0.06618604063987732, 0.10907616466283798, 0.53072512